In [1]:
# import sys
# print(sys.executable)
# print(sys.version)

In [2]:
#!pip install twython

The code in the notebook assumes that the file mentioned as value for variable `filename` is stored in a folder named "Twitter".

If your files are organised differently, change the values for values `dir` and `filename` below. 

Provide the details of your Twitter API in the cell below. 


In [3]:
import config_twitter

dir = 'Twitter'
filename = 'CharlieHebdo-ids-20150113.txt'


from twython import Twython
twitter = Twython(app_key= APP_KEY,
            app_secret= APP_SECRET ,
            oauth_token= OAUTH_TOKEN,
            oauth_token_secret= OAUTH_TOKEN_SECRET )


In [4]:
import sys
import time
import os
import re

import json
import zipfile
from math import ceil

twitter_ids = []

ids = open( filename )
for line in ids:
    twitter_ids.append(line.strip())
           
print(len(twitter_ids))         

6038370


In [ ]:
file_count = 1
removed = 0 

for file in os.listdir(dir):
    if re.search( r'txt$' , file ):
        print(file)
        no_ext = os.path.splitext(file)[0]
        count = int( re.split( r'_' , no_ext )[-1] )
        
        if file_count <= count:
            file_count = count + 1
            
        with open( os.path.join( dir, file )) as fh:
            for line in fh:
                if re.search( r'^TWITTER' , line ):
                    id = re.sub( r'^TWITTER\s+' , '' , line )
                    id = id.strip()
                    twitter_ids.remove(id)
                    removed += 1
                                
print(len(twitter_ids))  
print( f'{removed} ids removed' )

tweets-CharlieHebdo-ids-20150113.txt_1.txt
tweets-CharlieHebdo-ids-20150113.txt_2.txt
tweets-CharlieHebdo-ids-20150113.txt_3.txt
tweets-CharlieHebdo-ids-20150113.txt_7.txt
tweets-CharlieHebdo-ids-20150113.txt_6.txt
tweets-CharlieHebdo-ids-20150113.txt_4.txt
tweets-CharlieHebdo-ids-20150113.txt_5.txt
tweets-CharlieHebdo-ids-20150113.txt_8.txt


In [ ]:
from IPython.display import clear_output
import time


multilevel = ['entities' , 'user' , 'retweeted_status' , ]
ignore = [ 'user-profile_background_color',
          'user-profile_background_image_url',
          'user-profile_background_image_url_https',
          'user-profile_background_tile',
          'user-profile_image_url',
          'user-profile_image_url_https',
          'user-profile_banner_url',
          'user-profile_link_color',
          'user-profile_sidebar_border_color',
          'user-profile_sidebar_fill_color',
          'user-profile_text_color',
         'user-contributors_enabled', 
        'user-is_translator', 
        'user-is_translation_enabled', 
        'user-profile_use_background_image', 
        'user-has_extended_profile', 
        'user-default_profile', 
        'user-default_profile_image', 
        'user-following', 
        'user-follow_request_sent', 
        'user-notifications', 
        'user-translator_type' , 
         'user-entities' , 
         'user-withheld_in_countries']



n = 99


skip = []

log_file = 'log.txt'
if os.path.isfile('log_file'):
    with open(log_file) as fh:
        for line in log_file:
            skip.append(line.strip())
            
            
out_file = f"tweets-{filename}_{file_count}.txt" 
print( f'Downloaded tweets will be saved as {out_file}')
count_tweets = 0 

out = open( os.path.join( dir , out_file ), 'w' , encoding = 'utf-8')


total = len(twitter_ids)

while len(twitter_ids) > 0:
    ## navigate across all ids
    ## in sets of 99 
    look_up = twitter_ids[:n]
    twitter_ids = twitter_ids[n:]
    

        
    print('New batch')
    results = twitter.lookup_status(id= look_up )
    
    for dict_level1 in results:
        tweet = dict()
        count_tweets += 1
        hashtags = []
        user_mentions = []
        for key in dict_level1:
            if key not in multilevel:
                tweet[key] = dict_level1[key]
            else:

                if key == 'user':
                    for field in dict_level1[key]:
                        tweet[f'user-{field}'] = dict_level1[key][field]
                else:
                    dict_level2 = dict_level1[key]
                    for key2 in dict_level2:

                        if key2 == 'hashtags':
                            for key3 in dict_level2[key2]:
                                hashtags.extend( [ key3['text'] ] )
                        elif key2 == 'user_mentions':
                            for key3 in dict_level2[key2]:
                                user_mentions.extend( [ f"{key3.get('screen_name','')}; {key3.get('name','')}; {key3.get('id','')}" ] ) 

        tweet['hashtags'] = hashtags
        tweet['user_mentions'] = user_mentions
        out.write( f"TWITTER {tweet['id'] }\n" )
        for field in tweet:
            if field not in ignore:
                out.write( f"{field}: {tweet[field]}\n")
        out.write('\n\n')
        
        percentage = (count_tweets/total)*100
        print( f"{count_tweets} tweets ({ round(percentage,3) }%) downloaded ...")
    
        
        if count_tweets % 1000 == 0:
            time.sleep(60*2)
            print('Pausing to avoid 429 error (Too Many Requests) ...')
                

out.close()    